In [1]:
from numbapro import guvectorize
import numpy as np

numbapro:1: ImportWarning: The numbapro package is deprecated in favour of the accelerate package. Please update your code to use equivalent functions from accelerate.


In [2]:
@guvectorize(['void(int64[:,:],int64[:,:],int64[:,:])'], '(m,n),(n,p)->(m,p)')
def matmul(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]


dim = 10
A = np.random.randint(dim, size=(dim, dim))
B = np.random.randint(dim, size=(dim, dim))

C = matmul(A, B)

In [3]:
print("input matrix A:")
print(A)
print("input matrix B:")
print(B)
print("result matrix C = A*B:")
print(C)

input matrix A:
[[9 4 4 7 3 6 7 0 0 7]
 [0 4 7 0 2 2 8 7 5 5]
 [1 1 7 0 5 4 8 9 6 7]
 [7 3 7 8 2 0 5 3 6 1]
 [8 1 1 3 7 7 9 3 6 8]
 [7 9 7 2 3 3 1 8 2 1]
 [2 1 7 0 8 1 3 6 5 7]
 [8 3 8 2 6 8 6 0 6 4]
 [8 7 4 0 3 3 8 7 3 0]
 [5 8 9 8 4 7 8 7 3 4]]
input matrix B:
[[6 8 7 5 0 7 4 4 5 3]
 [5 1 8 1 9 2 0 0 0 1]
 [9 5 8 0 7 8 6 4 8 4]
 [9 1 6 8 4 1 8 5 2 3]
 [6 0 3 1 9 7 1 1 5 0]
 [2 1 0 9 4 0 8 5 1 1]
 [7 3 1 7 2 4 4 7 7 1]
 [0 3 3 5 9 7 7 9 1 1]
 [3 3 2 0 3 2 1 7 8 8]
 [0 5 5 3 6 3 3 0 1 8]]
result matrix C = A*B:
[[252 165 220 232 199 180 216 169 168 137]
 [170 126 158 130 235 184 161 194 176 129]
 [186 152 168 169 276 228 205 236 210 157]
 [257 149 214 157 187 189 186 202 204 140]
 [226 174 181 237 230 211 212 225 218 169]
 [205 143 232 140 263 213 178 177 145 100]
 [166 130 168 100 251 213 146 159 181 141]
 [265 173 212 191 231 218 208 204 240 159]
 [208 148 188 168 218 210 167 206 177  89]
 [326 182 286 267 343 258 294 275 231 161]]


In [5]:
import numbapro.cudalib.cublas as cublas
import numpy as np
from timeit import default_timer as timer

dim = 10


def gemm():
    print("Version 2".center(80, '='))
    A = np.random.rand(dim, dim)
    B = np.random.rand(dim, dim)
    D = np.zeros_like(A, order='F')

    print("matrix A:")
    print(A)
    print("matrix B: ")
    print(B)
    # numpy
    start = timer()
    E = np.dot(A, B)
    numpy_time = timer() - start
    print("numpy took %f seconds" % numpy_time)
    # cublas
    blas = cublas.Blas()

    start = timer()
    blas.gemm('T', 'T', dim, dim, dim, 1.0, A, B, 1.0, D)
    cuda_time = timer() - start
    print("result matrix evaluated with cublas")
    print(D)
    print("cublas took %f seconds" % cuda_time)
    diff = np.abs(D - E)
    print("maximum error %f" % np.max(diff))

if __name__ == '__main__':
    gemm()

===================================Version 2====================================
matrix A:
[[0.37983941 0.06237608 0.54510063 0.70222439 0.4730661  0.42359997
  0.52517894 0.49953445 0.5972997  0.17213909]
 [0.4722598  0.80614596 0.31502114 0.55756861 0.34068162 0.45830107
  0.57232948 0.54912753 0.97620826 0.86362018]
 [0.55772379 0.39599717 0.0304962  0.0514733  0.70660585 0.8662308
  0.87582431 0.3333349  0.51599078 0.82499977]
 [0.28407683 0.68183472 0.62255657 0.32697835 0.68274155 0.2071924
  0.96979512 0.01624247 0.06879704 0.26215085]
 [0.07742271 0.81009319 0.74997836 0.61639927 0.3455769  0.46495072
  0.00786886 0.7097451  0.47001787 0.56045781]
 [0.32898546 0.81586896 0.98116987 0.54775428 0.64995463 0.63106759
  0.06248405 0.16715187 0.03115152 0.63382168]
 [0.65361713 0.54759696 0.31605037 0.40342093 0.24111648 0.43066423
  0.54943777 0.59010837 0.23226709 0.55481336]
 [0.63866949 0.56595866 0.69055426 0.80399248 0.15083467 0.79875414
  0.11354159 0.01666308 0.90716922 0.8